## 作業目標: 了解斷詞演算法的背後計算

### 根據課程講述的內容, 請計算出下列剩餘所有情況的
若有一個人連續觀察到三天水草都是乾燥的(Dry), 則這三天的天氣機率為何？(可參考講義第13頁)
(Hint: 共有8種可能機率)

```python
states = ('sunny', 'rainy')
observations = ('dry', 'damp', 'soggy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}
```

```
觀察狀態 = 'dry', 'dry', 'dry'
Sunny, Sunny, Sunny: 0.4*(0.6)*0.6*(0.6)*0.6*(0.6) = 0.031104
Rainy, Sunny, Sunny: 0.6*(0.1)*0.3*(0.6)*0.6*(0.6) = 0.003888

Sunny, Rainy, Sunny: 0.4*(0.6)*0.4*(0.1)*0.3*(0.6) = 0.001728
Rainy, Rainy, Sunny: 0.6*(0.1)*0.7*(0.1)*0.3*(0.6) = 0.000756

Sunny, Sunny, Rainy: 0.4*(0.6)*0.6*(0.6)*0.4*(0.1) = 0.003456
Rainy, Sunny, Rainy: 0.6*(0.1)*0.3*(0.6)*0.4*(0.1) = 0.000432

Sunny, Rainy, Rainy: 0.4*(0.6)*0.4*(0.1)*0.7*(0.1) = 0.000672
Rainy, Rainy, Rainy: 0.6*(0.1)*0.7*(0.1)*0.7*(0.1) = 0.000294

最大機率為: Sunny, Sunny, Sunny
```

### 根據上述條件, 寫出Viterbi應用程式

In [1]:
observations = ('dry', 'dry', 'dry') #實際上觀察到的狀態為dry, dry, dry
states = ('sunny', 'rainy')
start_probability = {'sunny': 0.4, 'rainy': 0.6}
transition_probability = {'sunny':{'sunny':0.6, 'rainy':0.4},
                          'rainy': {'sunny':0.3, 'rainy':0.7}}
emission_probatility = {'sunny': {'dry':0.6, 'damp':0.3, 'soggy':0.1},
                        'rainy': {'dry':0.1, 'damp':0.4, 'soggy':0.5}}

In [2]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        path[y] = start_p[y] * emit_p[y][obs[0]]

    # reversely sort path based on the probability of predicted states
    path = dict(sorted(path.items(), key=lambda item: item[1], reverse = True))

    proj_state = list(path.keys())[0] # projected state
    proj_prob = list(path.values())[0] # projected probability of proj_state

    # return the projected state with the largest possibility, eg. V = [{'sunny': 0.24}]
    V[0][proj_state] = proj_prob
    # update the path containing only the projected daily state, eg. path = {'sunny': ['sunny']}
    path = {proj_state: [proj_state]}

    
    # Run Viterbi for t > 0
    for t in range(1,len(obs)):
    
        # get the exact transition prob based on the previous projected state
        nextday_trans = trans_p[proj_state]

        newpath = {} # initialise a new path
        for y in states:
            newpath[y] = nextday_trans[y]*emit_p[y][obs[t]] # (transition prob) * (emission prob)

        # if the calculation hasn't reached the end of the obs, calculate the probability of the path and choose the best path
        # else, leave the decision, i.e. which path to choose, to the end of code
        if t < (len(obs) - 1):

            # reversely sort path based on the probability of predicted states
            newpath = dict(sorted(newpath.items(), key=lambda item: item[1], reverse = True))

            new_proj_state = list(newpath.keys())[0] # projected state
            new_proj_prob = proj_prob * list(newpath.values())[0] # projected probability of proj_state

            # return the projected state with the largest possibility, eg. V = [{'sunny': 0.24}]
            V.append({new_proj_state: new_proj_prob})
            # update the path containing only the projected daily state, eg. path = {'sunny': ['sunny']}
            newpath = {new_proj_state: list(path.values())[0]}
            newpath[new_proj_state].append(new_proj_state)

            # update the results
            path = newpath
            proj_state = new_proj_state
            proj_prob = new_proj_prob

        else:
            V.append(newpath)
            previously_predicted_path = list(path.values())[0]
            for y in states:
                V[len(obs) - 1][y] *= proj_prob
                path[y] = previously_predicted_path + [y]
    
    (prob, state) = max([(V[len(obs) - 1][final_state], final_state) for final_state in states])
    return (prob, path[state])

In [3]:
result = viterbi(observations,
                 states,
                 start_probability,
                 transition_probability,
                 emission_probatility)

In [4]:
result

(0.031103999999999996, ['sunny', 'sunny', 'sunny'])

In [5]:
# sample answer
# result